
# Joining Cellartracker Metadata with Sample Tracker

The idea of this project is to prototype getting metadata for my wines, and for a list from the sample tracker table, inner join to the sample tracker table, then perform EDA on the join.

First, get the sample tracker table:

In [1]:
import os

import pandas as pd

import numpy as np

os.getcwd()

'/Users/jonathan/wine_analysis_hplc_uv/notebooks'

In [2]:
from google_sheets_api import get_sheets_values_as_df

def sample_tracker_df_builder():

    df = get_sheets_values_as_df(
        spreadsheet_id='15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY',
        range='sample_tracker!A1:H200',
        creds_parent_path=os.path.join(os.getcwd(),'credentials_tokens'),
        )
    return df

sample_tracker_df = sample_tracker_df_builder()

print(sample_tracker_df.columns)

sample_tracker_df = sample_tracker_df[['id','vintage', 'name', 'size', 'open_date', 'sample_date', 'variety', 'notes']]

import dtale

dtale.show(sample_tracker_df)

Index(['id', 'vintage', 'name', 'sample_date', 'open_date', 'notes', 'variety',
       'size'],
      dtype='object')


So we should be able to join on the name alone, through the use of a fuzzy match. But first, let's get the cellartracker table.

In [3]:
from cellartracker import cellartracker

def get_cellar_tracker_table():

    client = cellartracker.CellarTracker('OctaneOolong', 'S74rg4z3r1')

    usecols = ['Size', 'Vintage', 'Wine', 'Locale', 'Country', 'Region', 'SubRegion', 'Appellation', 'Producer', 'Type', 'Color', 'Category', 'Varietal']

    cellar_tracker_df = pd.DataFrame(client.get_list())

    cellar_tracker_df = cellar_tracker_df[usecols]

    # clean it up. lower values and columns, replace 1001 with nv, check datatypes

    cellar_tracker_df = cellar_tracker_df.apply(lambda x : x.str.lower() if str(x) else x)
    cellar_tracker_df.columns = cellar_tracker_df.columns.str.lower()
    cellar_tracker_df = cellar_tracker_df.rename({'wine' : 'name'}, axis = 1)

    cellar_tracker_df = cellar_tracker_df.replace({'1001' : 'nv'})

    return cellar_tracker_df
    
cellar_tracker_df = get_cellar_tracker_table()

dtale.show(cellar_tracker_df)

2023-03-28 15:57:31,189 - INFO     - Connecting to https://www.cellartracker.com/xlquery.asp
2023-03-28 15:57:32,069 - INFO     - Successfully connected to https://www.cellartracker.com/xlquery.asp


That was easy! Now to join them. We should form a join col made up of the vintage and name in both tables.

According to chatgpt, "Pr&#333;ter&#333;" instead of "prōterō" is due to being encoded as a 'html character entity', and I can use the HTML module to 'unescape' the string.

In [4]:
import html

def unescape_html(s):
    return html.unescape(s)

cellar_tracker_df = cellar_tracker_df.applymap(unescape_html)

dtale.show(cellar_tracker_df)

In [5]:
def form_join_col(df):
    df['join_key'] = df['vintage'] + " " + df['name']
    return df

sample_tracker_df = form_join_col(sample_tracker_df)
cellar_tracker_df = form_join_col(cellar_tracker_df)
sample_tracker_df

,id,vintage,name,size,open_date,sample_date,variety,notes,join_key
0,z1,2016,zema estate 'family selection' cabernet sauvignon,750,,,cabernet-sauvignon,"fridge. sampled at 21:20 20230122, stored for ...",2016 zema estate 'family selection' cabernet s...
1,z2,2016,zema estate 'family selection' cabernet sauvignon,750,,,cabernet-sauvignon,"freezer storage. sampled at 21:20 20230122, st...",2016 zema estate 'family selection' cabernet s...
2,z3,2016,zema estate 'family selection' cabernet sauvignon,750,,,cabernet-sauvignon,ambient storage. sampled at 21:20 20230122.,2016 zema estate 'family selection' cabernet s...
3,1,2022,william downie 'cathedral' pinot noir,750,,,pinot-noir,ambient 2 weeks. sampled 20230111.,2022 william downie 'cathedral' pinot noir
4,2,2021,babo chianti,750,,,sangiovese,ambient 2 weeks. sampled 20230111.,2021 babo chianti
...,...,...,...,...,...,...,...,...,...
68,67,2019,wynns coonawarra estate shiraz,750,,2023-03-21,,,2019 wynns coonawarra estate shiraz
69,68,2014,shaw + smith shiraz,750,2023-03-21,2023-03-23,,,2014 shaw + smith shiraz
70,69,2021,domenica chardonnay,750,2023-03-23,2023-03-23,,,2021 domenica chardonnay
71,70,2020,hochkirch pinot noir,750,2023-03-23,2023-03-23,,,2020 hochkirch pinot noir


In [6]:
def join_dfs_with_fuzzy(df1, df2):
    from fuzzywuzzy import fuzz, process

    def fuzzy_match(s1, s2):
        return fuzz.token_set_ratio(s1, s2)
    
    df1['join_key_match'] = df1['join_key'].apply(lambda x: process.extractOne(x, df2['join_key'], scorer = fuzzy_match))

    # the above code produces a tuple of: ('matched_string', 'match score', 'matched_string_indice'). Usually it's two return values, but using scorer=fuzzy.token_sort_ratio or scorer=fuzz.token_set_ratio returns the index as well.

    df1['join_key_matched'] = df1['join_key_match'].apply(lambda x: x[0])
    df1['join_key_similarity'] = df1['join_key_match'].apply(lambda x : x[1])

    print(df1['join_key_similarity'][0])

    df1.drop(columns = ['join_key_match'], inplace = True)

    merged_df = pd.merge(df1, df2, left_on='join_key_matched', right_on='join_key')

    return merged_df

df = join_dfs_with_fuzzy(sample_tracker_df, cellar_tracker_df)

# original attempt fails on: 'null mystery', '2021 criante c21 cataratto', '2020 dom. simha paysan fleur sauvage', '2021 billaud d'or bourgogne blanc', 'null empty id, missing wine', '2021 st hugo gsm', '? yangarra 'roux beaute' roussanne'.

# I now want to have a look at all the incorrect matches and get theirx similarity scores. This should be simple enough, just need another column to keep the sim scores.

pd.options.display.min_rows = 50


dtale.show(df[['join_key_similarity','join_key_x', 'join_key_y']].sort_values('join_key_similarity'))

100


From this, a similarity score below 55 is when the matches became false. Thus a simple fix:

In [7]:
def join_dfs_with_fuzzy(df1, df2):
    from fuzzywuzzy import fuzz, process

    def fuzzy_match(s1, s2):
        return fuzz.token_set_ratio(s1, s2)
    
    df1['join_key_match'] = df1['join_key'].apply(lambda x: process.extractOne(x, df2['join_key'], scorer = fuzzy_match))

    # the above code produces a tuple of: ('matched_string', 'match score', 'matched_string_indice'). Usually it's two return values, but using scorer=fuzzy.token_sort_ratio or scorer=fuzz.token_set_ratio returns the index as well.

    df1['join_key_matched'] = df1['join_key_match'].apply(lambda x: x[0] if x[1] > 65 else None)
    df1['join_key_similarity'] = df1['join_key_match'].apply(lambda x : x[1] if x[1] > 65 else None)

    df1.drop(columns = ['join_key_match'], inplace = True)

    merged_df = pd.merge(df1, df2, left_on='join_key_matched', right_on='join_key', how = 'left')

    return merged_df

df = join_dfs_with_fuzzy(sample_tracker_df, cellar_tracker_df)

dtale.show(df[['join_key_similarity','join_key_x', 'join_key_y']].sort_values('join_key_similarity'))

/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages/dtale/views.py:755: FutureWarning:

['join_key_y'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



In [8]:
def df_cleanup(df):

    df = df.drop(['vintage_x', 'name_x', 'join_key_x', 'join_key_matched', 'join_key_y', 'variety', 'size_y'], axis = 1)

    df = df.rename({
                    'size_x' : 'size',
                    'vintage_y' : 'vintage',
                    'name_y' : 'name'
                    }, axis = 1)
    
    ordered_cols = ['id', 'vintage', 'name', 'size']

    rest_of_cols = list(df.columns.drop(ordered_cols))

    ordered_cols = ordered_cols + rest_of_cols
    
    df = df[ordered_cols]
    
    display(df)

df_cleanup(df)

,id,vintage,name,size,open_date,sample_date,notes,join_key_similarity,locale,country,region,subregion,appellation,producer,type,color,category,varietal
0,z1,2016,zema estate cabernet sauvignon family selection,750,,,"fridge. sampled at 21:20 20230122, stored for ...",100.0,"australia, south australia, limestone coast, c...",australia,south australia,limestone coast,coonawarra,zema estate,red,red,dry,cabernet sauvignon
1,z2,2016,zema estate cabernet sauvignon family selection,750,,,"freezer storage. sampled at 21:20 20230122, st...",100.0,"australia, south australia, limestone coast, c...",australia,south australia,limestone coast,coonawarra,zema estate,red,red,dry,cabernet sauvignon
2,z3,2016,zema estate cabernet sauvignon family selection,750,,,ambient storage. sampled at 21:20 20230122.,100.0,"australia, south australia, limestone coast, c...",australia,south australia,limestone coast,coonawarra,zema estate,red,red,dry,cabernet sauvignon
3,1,2022,william downie cathedral,750,,,ambient 2 weeks. sampled 20230111.,100.0,"australia, victoria, central victoria, upper g...",australia,victoria,central victoria,upper goulburn,william downie,red,red,dry,pinot noir
4,2,2021,babo chianti,750,,,ambient 2 weeks. sampled 20230111.,100.0,"italy, tuscany, chianti",italy,tuscany,chianti,chianti,babo,red,red,dry,sangiovese
5,,2021,joshua cooper cabernet sauvignon landsborough ...,750,2023-02-04,,,100.0,"australia, victoria, western victoria, pyrenees",australia,victoria,western victoria,pyrenees,joshua cooper,red,red,dry,cabernet sauvignon
6,5,2022,william downie cathedral,750,02-02,,,100.0,"australia, victoria, central victoria, upper g...",australia,victoria,central victoria,upper goulburn,william downie,red,red,dry,pinot noir
7,6,2021,babo chianti,750,04-02,,,100.0,"italy, tuscany, chianti",italy,tuscany,chianti,chianti,babo,red,red,dry,sangiovese
8,7,2020,boutinot uva non grata,750,04-02,,,86.0,"france, vin de france",france,france,unknown,vin de france,boutinot,red,red,dry,gamay
9,8,2021,matias riccitelli malbec hey malbec!,750,04-02,,,100.0,"argentina, mendoza, lujan de cuyo",argentina,mendoza,lujan de cuyo,unknown,matias riccitelli,red,red,dry,malbec


Ok looking good. Now to chuck it all into a google sheet.

Let's call the sheet..

sample_overview

In [9]:
from pathlib import Path

sheet_url = 'https://docs.google.com/spreadsheets/d/15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY/edit#gid=347137817'
 

sheet_id = Path(sheet_url).parent.name

sheet_title = 'sample_overview'

creds_parent_path = Path.cwd() / 'credentials_tokens'

def make_result_sheet(sheet_id, sheet_title, creds_parent_path):

    from google_sheets_api import post_new_sheet

    sheet_title = 'sample_overview'

    creds_parent_path = Path.cwd() / 'credentials_tokens'

    response = post_new_sheet(sheet_id, sheet_title, creds_parent_path)

    display(response)

Now to post the results into the results sheet.

In [10]:
def post_results(df, sheet_id, range, creds_parent_path):
    from google_sheets_api import post_df_as_sheet_values

    response = post_df_as_sheet_values(df, sheet_id, range, creds_parent_path)

    print(response)

#post_results(df, sheet_id, f"{sheet_title}!A1", creds_parent_path)

2023-03-28 15:57:32,619 - INFO     - file_cache is only supported with oauth2client<4.0.0


{'spreadsheetId': '15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY', 'updatedRange': 'sample_overview!A1:Y74', 'updatedRows': 74, 'updatedColumns': 25, 'updatedCells': 1850}


2023-03-28 16:57:32,391 - INFO     - Executing shutdown due to inactivity...
2023-03-28 16:57:32,595 - INFO     - Executing shutdown...
2023-03-28 16:57:32,596 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonathan/.pyenv/versions/3.11.1/lib/python3.11/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_fun

Once this is done, the next thing to do is to identify all samples which have been run on the avantor 10cm column. The most straightforward method would be to go back to your notes to check when you swapped over to it, since you haven't used another column since.

On checking, it appears that the 10cm column was installed on the 02/02. Now, from the table above, some of the wines only have a recorded open date, and some only have a recorded sample date.

To verify the sampling dates, we should go back to Agilette and see if we can form a table with the sample 'id' as the primary key, then join on that.

Regardless, looks like there are a lot of samples dated from after the 2nd. That being said, I think a run of samples before the 20's were all compromised by time before they were run. Definitely need to review the experiments.

That's enough in this note. Will continue in [the next notebook](./2023-03-28_tabulating-past-experiments.ipynb)